In [1]:
import os
import btagging as btag
reload(btag)

Welcome to ROOTaaS 6.06/05


<module 'btagging' from '/mnt/t3nfs01/data01/shome/musella/Analysis/jupyter/btagging/python/btagging.pyc'>

In [2]:
ntuples = 'rFlavPhaseI/v1'

%env data=$btag.IO.ldata/$ntuples
folders = ! ls $data | sort -t_ -k 3 -n

env: data=/mnt/t3nfs01/data01/shome/musella/store/t2/btag/rFlavPhaseI/v1


In [3]:
folders

['TT_TuneCUETP8M2T4_13TeV-powheg-pythia8',
 'QCD_Pt_15to30_TuneCUETP8M1_13TeV_pythia8',
 'QCD_Pt_30to50_TuneCUETP8M1_13TeV_pythia8',
 'QCD_Pt_50to80_TuneCUETP8M1_13TeV_pythia8',
 'QCD_Pt_80to120_TuneCUETP8M1_13TeV_pythia8',
 'QCD_Pt_120to170_TuneCUETP8M1_13TeV_pythia8',
 'QCD_Pt_170to300_TuneCUETP8M1_13TeV_pythia8',
 'QCD_Pt_300to470_TuneCUETP8M1_13TeV_pythia8',
 'QCD_Pt_470to600_TuneCUETP8M1_13TeV_pythia8']

In [4]:
from joblib import Parallel, delayed

# columns = ["Ev_event","Ev_run","Ev_lumi","Jet_pt","Jet_index","Jet_eta",
#                        "Jet_genPt","Jet_genFlavourCleaned","Jet_genPartonFlavour"]

# where="Jet_index>=0 && Cand_index == -1"

folders = ['QCD_Pt_15to30_TuneCUETP8M1_13TeV_pythia8']

columns = None
where = '1'

def make_dataset(iproc,folder,store=True,**kwargs):
    print('reading %s' % folder)
    weight,df = btag.IO.load_input_ntuples(os.path.join(ntuples,folder),
                                           columns=columns,where=where,**kwargs)
    print('defining process %s' % folder)
    btag.preprocessing.define_process_weight(df,iproc,weight)
    print('adjusting %s' % folder)
    btag.preprocessing.adjust_and_compress(df,doFloat16=False)
    
    if store:
        print('storing %s' % folder)
        fname = '/scratch/musella/%s.root' % folder
        df.to_root(fname,folder)
        fname = '/scratch/musella/%s.hd5' % folder
        df.to_hdf(fname,folder,compression=9,complib='bzip2',mode='w')
        return fname
    return df

qcd_folders = filter(lambda x: x.startswith('QCD'), folders)
qcd = Parallel(n_jobs=8,verbose=60)(delayed(make_dataset)(iproc,folder,nevents=100000) for iproc, folder in enumerate(qcd_folders))

# top_folders = filter(lambda x: x.startswith('TT'), folders)
# top = Parallel(n_jobs=5,verbose=60)(delayed(make_dataset)(iproc,folder,nevents=100000) for iproc, folder in enumerate(top_folders))

reading QCD_Pt_15to30_TuneCUETP8M1_13TeV_pythia8
(505731.0, 1824000000.0)
defining process QCD_Pt_15to30_TuneCUETP8M1_13TeV_pythia8
adjusting QCD_Pt_15to30_TuneCUETP8M1_13TeV_pythia8
storing QCD_Pt_15to30_TuneCUETP8M1_13TeV_pythia8
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:  4.5min
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:  4.5min finished


In [ ]:
qcd_df = qcd.pop(0)
while len(qcd) > 0:
    qcd_df = qcd_df.append(qcd.pop(0))

In [ ]:
top_df = top.pop(0)

In [ ]:
# qcd_df.to_hdf('/scratch/musella/qcd.hd5','qcd',compression=9,complib='bzip2',mode='w')
# top_df.to_hdf('/scratch/musella/top.hd5','top',compression=9,complib='bzip2',mode='w')

In [ ]:
! ls -lh /scratch/musella/qcd.root /scratch/musella/top.root 

In [ ]:
import numpy as np 



In [ ]:
adjust_and_compress(qcd_df).to_hdf('/scratch/musella/qcd_all_br_flt16.hd5','qcd',compression=9,complib='bzip2',mode='w')

In [ ]:
# %matplotlib inline
# from matplotlib.colors import LogNorm
# plt.figure(figsize=(10,10))
# counts, xedges, yedges, values = plt.hist2d(x=qcd_df.Jet_pt,y=qcd_df.Jet_genFlavourCleaned,weights=qcd_df.weight,
#            range=[[20,100],[0.5,21.5]],bins=[80,21],norm=LogNorm(),cmap='PuBu');
# plt.show()

In [ ]:
# import numpy as np

# gby = qcd_df.groupby('Jet_genFlavourCleaned')
# groups = gby.groups

In [ ]:
# gby.hist(column='Jet_pt',weights='weight',figsize=(10,10),alpha=0.1,bins=np.linspace(20,100,80))#,layout=(3,3));

In [ ]:
# gby.hist(column='Jet_genPartonFlavour',weights='weight',figsize=(10,10),alpha=0.1)#,layout=(3,3));

In [ ]:
# ddf = qcd_df[['Jet_pt','Jet_eta','weight']]

# for gp,idx in groups.iteritems():
#     fig = plt.figure(figsize=(7,7))
#     df = ddf.loc[idx]
#     plt.hist2d(x=df.Jet_pt,y=np.abs(df.Jet_eta),weights=df.weight,
#            range=[[20,100],[0,2.5]],bins=[80,25],norm=LogNorm(),cmap='PuBu')
#     plt.show()


In [ ]:
# gby2 = qcd_df.groupby( ['Jet_label','proc'] )

In [ ]:
# gby2.hist(column='Jet_pt',weights='weight',figsize=(10,10),alpha=0.1,bins=np.linspace(20,100,80))#,layout=(3,3));

In [ ]:
from sklearn import ensemble


In [ ]:
clf = ensemble.GradientBoostingClassifier(max_depth=5,learning_rate=0.2,n_estimators=100,min_weight_fraction_leaf=1e-4)

In [ ]:
X = qcd_df[['Jet_pt','Jet_eta','Jet_label']]
y = qcd_df.proc.ravel()
w = qcd_df.weight.ravel()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(categorical_features=[2])

In [ ]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(encoder,clf)

In [ ]:
pipeline.fit(X,y,gradientboostingclassifier__sample_weight=w)

In [ ]:
from gzip import open as gopen

import cPickle as pickle

with gopen('/scratch/musella/sampler.pkl.gz','w+') as fout:
    pickle.dump(pipeline,fout)
    fout.close()
